In [1]:
import sys

# sys.path.append('../')
import GTPyhop.gtpyhop as gtpyhop
import pddlgym



Imported GTPyhop version 1.0.
Messages from find_plan will be prefaced with 'FP>'.
Messages from run_lazy_lookahead will be prefaced with 'RLL>'.


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
PATH_TO_GYM = "pddlgym/"
domain_name = "snake"
problem_name = "PDDLEnvSnake-v0"
domain = gtpyhop.Domain(domain_name)
domain

Domain('snake', _action_dict={}, _command_dict={}, _task_method_dict={'_verify_g': [<function _m_verify_g at 0x1041a16c0>], '_verify_mg': [<function _m_verify_mg at 0x1041a1750>]}, _unigoal_method_dict={}, _multigoal_method_list=[])

In [8]:
'''
Define rigid relations for the domain
'''

rigid = gtpyhop.State('rigid relations')

# snake domain is only coordinates/fields according to problem size
# p01/pddl is a 5x5 grid, so coords are (0,0) to (4,4)

size = 5
coords = []
for i in range(size):
    for j in range(size):
        coord = f'pos{i}-{j}'
        coords.append(coord)

coords.append('dummypoint')

rigid.types = {
    'coord': coords,
}

# Helper
def is_a(variable,type):
    """
    In most classical planners, one would declare data-types for the parameters
    of each action, and the data-type checks would be done by the planner.
    GTPyhop doesn't have a way to do that, so the 'is_a' function gives us a
    way to do it in the preconditions of each action, command, and method.
    
    'is_a' doesn't implement subtypes (e.g., if rigid.type[x] = y and
    rigid.type[x] = z, it doesn't infer that rigid.type[x] = z. It wouldn't be
    hard to implement this, but it isn't needed in the simple-travel domain.
    """
    return variable in rigid.types[type]

rigid

State('rigid relations', types={'coord': ['pos0-0', 'pos0-1', 'pos0-2', 'pos0-3', 'pos0-4', 'pos1-0', 'pos1-1', 'pos1-2', 'pos1-3', 'pos1-4', 'pos2-0', 'pos2-1', 'pos2-2', 'pos2-3', 'pos2-4', 'pos3-0', 'pos3-1', 'pos3-2', 'pos3-3', 'pos3-4', 'pos4-0', 'pos4-1', 'pos4-2', 'pos4-3', 'pos4-4', 'dummypoint']})

In [13]:
rigid.types['coord']

['pos0-0',
 'pos0-1',
 'pos0-2',
 'pos0-3',
 'pos0-4',
 'pos1-0',
 'pos1-1',
 'pos1-2',
 'pos1-3',
 'pos1-4',
 'pos2-0',
 'pos2-1',
 'pos2-2',
 'pos2-3',
 'pos2-4',
 'pos3-0',
 'pos3-1',
 'pos3-2',
 'pos3-3',
 'pos3-4',
 'pos4-0',
 'pos4-1',
 'pos4-2',
 'pos4-3',
 'pos4-4',
 'dummypoint']

In [12]:
# define initial state for p01
if problem_name == "PDDLEnvSnake-v0":
    initial = gtpyhop.State('state0')

    # point adjacency relations
    isadjacent = {'dummypoint': []}

    for i in range(size):
        for j in range(size):
            coord = f'pos{i}-{j}'
            adjacent_coords = []
            # up
            if i > 0:
                adjacent_coords.append(f'pos{i-1}-{j}')
            # down
            if i < size - 1:
                adjacent_coords.append(f'pos{i+1}-{j}')
            # left
            if j > 0:
                adjacent_coords.append(f'pos{i}-{j-1}')
            # right
            if j < size - 1:
                adjacent_coords.append(f'pos{i}-{j+1}')
            isadjacent[coord] = adjacent_coords

    initial.isadjacent = isadjacent

    # snake spawn + point spawns
    # set initial tail position
    tailsnake  = 'pos3-0'
    initial.tailsnake = {tailsnake: True} | {coord: False for coord in rigid.types['coord'] if coord != tailsnake}

    # set initial head position
    headsnake = 'pos4-0'
    initial.headsnake = {headsnake: True} | {coord: False for coord in rigid.types['coord'] if coord != headsnake}

    # initial nextsnake position
    nextsnake = {headsnake: tailsnake}
    initial.nextsnake = nextsnake | {coord: None for coord in rigid.types['coord'] if coord not in nextsnake.keys()}

    # set blocked positions (obstacles + snake spawnpoints)
    # NOTE: no obstacles in p01
    blocked_pos = []
    blocked_pos.extend([tailsnake, headsnake])
    initial.blocked = {coord: True for coord in blocked_pos} | {coord: False for coord in rigid.types['coord'] if coord not in blocked_pos}
    
    # set initial food spawn
    spawn = 'pos2-0'
    initial.spawn = {spawn: True} | {coord: False for coord in rigid.types['coord'] if coord != spawn}

    # nextspawn relations
    nextspawn = {
        'pos1-0' :'dummypoint',
        'pos2-0' :'pos1-4',
        'pos1-4' :'pos1-1',
        'pos1-1' :'pos0-1',
        'pos0-1' :'pos3-3',
        'pos3-3' :'pos4-2',
        'pos4-2' :'pos3-4',
        'pos3-4' :'pos0-0',
        'pos0-0' :'pos1-2',
        'pos1-2' :'pos1-0'
    }

    initial.nextspawn = nextspawn | {coord: None for coord in rigid.types['coord'] if coord not in nextspawn.keys()}

    # ispoint
    ispoint = ['pos0-4', 'pos3-1', 'pos1-3', 'pos2-4', 'pos4-1']
    initial.ispoint = {coord: True for coord in ispoint} | {coord: False for coord in rigid.types['coord'] if coord not in ispoint}

else:
    raise NotImplementedError("Only PDDLEnvSnake-v0 is implemented")

In [14]:
initial.headsnake

{'pos4-0': True,
 'pos0-0': False,
 'pos0-1': False,
 'pos0-2': False,
 'pos0-3': False,
 'pos0-4': False,
 'pos1-0': False,
 'pos1-1': False,
 'pos1-2': False,
 'pos1-3': False,
 'pos1-4': False,
 'pos2-0': False,
 'pos2-1': False,
 'pos2-2': False,
 'pos2-3': False,
 'pos2-4': False,
 'pos3-0': False,
 'pos3-1': False,
 'pos3-2': False,
 'pos3-3': False,
 'pos3-4': False,
 'pos4-1': False,
 'pos4-2': False,
 'pos4-3': False,
 'pos4-4': False,
 'dummypoint': False}

In [22]:
initial.spawn

{'pos2-0': True,
 'pos0-0': False,
 'pos0-1': False,
 'pos0-2': False,
 'pos0-3': False,
 'pos0-4': False,
 'pos1-0': False,
 'pos1-1': False,
 'pos1-2': False,
 'pos1-3': False,
 'pos1-4': False,
 'pos2-1': False,
 'pos2-2': False,
 'pos2-3': False,
 'pos2-4': False,
 'pos3-0': False,
 'pos3-1': False,
 'pos3-2': False,
 'pos3-3': False,
 'pos3-4': False,
 'pos4-0': False,
 'pos4-1': False,
 'pos4-2': False,
 'pos4-3': False,
 'pos4-4': False,
 'dummypoint': False}

In [ ]:
# Actions

# NOTE: not bothering to check types since everything is_a coord

def move(state, head, newhead, tail, newtail):
    
    if state.headsnake[head] and \
       newhead in state.isadjacent[head] and \
       state.tailsnake[tail] and \
       state.nextsnake[newtail] == tail and \
       not state.blocked[newhead] and \
       not state.ispoint[newhead]:

        state.blocked[newhead] = True
        state.headsnake[newhead] = True
        state.nextsnake[newhead] = head
        state.headsnake[head] = False
        state.blocked[tail] = False
        state.tailsnake[tail] = False
        state.nextsnake[newtail] = None
        state.tailsnake[newtail] = True

        return state


# eat food and spawn new food
def move_and_eat_spawn(state, head, newhead, spawnpoint, nextspawnpoint):
    
    if state.headnsnake[head] and \
       newhead in state.isadjacent[head] and \
       not state.blocked[newhead] and \
       state.ispoint(newhead) and \
       state.spawn[spawnpoint] and \
       state.nextspawn[spawnpoint] == nextspawnpoint and \
       spawnpoint != 'dummypoint':
        
        state.blocked[newhead] = True
        state.headsnake[newhead] = True
        state.nextsnake[newhead] = head
        state.headsnake[head] = False
        state.ispoint[newhead] = False
        state.ispoint[spawnpoint] = True
        state.spawn[spawnpoint] = False
        state.spawn[nextspawnpoint] = True

        return state

# eat food but no new spawn
def move_and_eat_no_spawn(state, head, newhead):
    
    if state.headsnake[head] and \
       newhead in state.isadjacent[head] and \
       not state.blocked[newhead] and \
       state.ispoint[newhead] and \
       state.spawn['dummypoint']:
        
        state.blocked[newhead] = True
        state.headsnake[newhead] = True
        state.nextsnake[newhead] = head
        state.headsnake[head] = False
        state.ispoint[newhead] = False

        return state

In [ ]:
import simple

In [6]:
literal = initial.ispoint
print(len(literal))
literal

26


{'pos0-4': True,
 'pos3-1': True,
 'pos1-3': True,
 'pos2-4': True,
 'pos4-1': True,
 'pos0-0': False,
 'pos0-1': False,
 'pos0-2': False,
 'pos0-3': False,
 'pos1-0': False,
 'pos1-1': False,
 'pos1-2': False,
 'pos1-4': False,
 'pos2-0': False,
 'pos2-1': False,
 'pos2-2': False,
 'pos2-3': False,
 'pos3-0': False,
 'pos3-2': False,
 'pos3-3': False,
 'pos3-4': False,
 'pos4-0': False,
 'pos4-2': False,
 'pos4-3': False,
 'pos4-4': False,
 'dummypoint': False}